## Generate Subjective Value (SV) difference

Use the BHM generated parameters to generate SV difference


### Import libraries and modules

In [1]:
import pandas as pd
import os

In [2]:
idm_dir = '/Volumes/UCDN/datasets/IDM/'
agg_fn = os.path.join(idm_dir,'Aggregate_All_Model_Parameters.csv')
df_agg = pd.read_csv(agg_fn)
df_agg



,subject,cdd_meta,cdd_meta_nLL,cdd_CC1,cdd_CC2,cdd_NLH_meta,cdd_NLH_meta_nLL,cdd_NLH_CC1,cdd_NLH_CC2,crdm_meta,...,cdd_NLH_gamma,cdd_NLH_kappa,cdd_NLH_AIC,cdd_NLH_BIC,cdd_NLH_R2,crdm_gamma_bhm,crdm_beta_bhm,crdm_alpha_bhm,cdd_gamma_bhm,cdd_kappa_bhm
0,23_IDM_0001,2.136203,57.118670,2.294122e+00,2.345802e+00,2.086869,58.591148,1.342594e+00,2.701736e+00,1.667265,...,0.097745,0.008722,49.324260,54.432013,0.655847,0.825023,0.472010,1.236226,0.322426,0.006589
1,23_IDM_0002,5.674360,43.754399,2.294902e-08,2.294902e-08,4.753734,30.703390,6.364065e-09,6.364064e-09,0.995587,...,8.000000,0.004870,69.744891,74.852645,0.500790,5.490793,0.751427,0.147275,0.189111,0.073521
2,23_IDM_0003,1.168343,60.976375,2.041643e+00,4.246610e+00,0.770910,60.528494,1.537380e+00,2.962159e+00,0.160503,...,1.597424,0.011211,38.749071,43.856825,0.736146,1.869803,0.264056,0.708625,0.690407,0.017571
3,23_IDM_0004,2.573175,81.525026,2.673312e+01,2.505079e+01,1.077842,81.187400,5.899701e+00,4.975314e+00,6.745691,...,1.966599,0.025623,74.280186,79.408883,0.471912,2.320312,-0.366157,0.464319,0.307844,0.086223
4,23_IDM_0005,9.043006,28.329300,4.692423e+00,8.410461e-01,3.255872,31.038921,1.684785e+00,4.325451e-01,1.744263,...,3.234854,0.004594,27.704761,32.812514,0.820007,1.720650,0.467945,0.557125,0.917327,0.008758
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144,23_IDM_0145,0.638421,52.110183,5.719150e-01,3.264194e+00,0.227142,45.263124,5.927569e-01,2.776947e+00,0.784659,...,3.765005,0.002200,37.002454,42.131150,0.752018,3.720521,0.507791,0.473570,0.452490,0.003297
145,23_IDM_0146,0.298664,6.647347,2.144200e+00,2.280526e+00,1.468534,4.696306,1.528960e-07,1.528961e-07,0.253394,...,2.790440,0.002200,19.201326,24.330023,0.885777,1.595158,0.006735,0.684302,1.489067,0.000840
146,23_IDM_0147,9.999990,81.126963,9.844003e+00,8.196444e+00,9.999999,80.995398,4.476188e+00,3.052697e+00,2.352597,...,5.918199,0.018036,51.196730,56.325427,0.645362,5.641400,0.890332,0.263881,0.360768,0.124497
147,23_IDM_0148,1.520627,83.271773,5.732734e+00,8.478175e+00,1.232994,82.758779,4.282084e+00,5.630210e+00,9.999882,...,1.099538,0.023987,66.613269,71.741966,0.529522,1.005348,0.706722,0.592807,0.311055,0.050659
